### LitCoin NLP Challenge: Part 1

*This 2-phase competition is part of the NASA Tournament Lab and hosted by NCBI (The National Center for Biotechnology Information), NCATS (The National Center for Advancing Translational Sciences) and NIH (National Institutes of Health). These institutions, in collaboration with bitgrit and CrowdPlat, have come together to bring this challenge where one can deploy their data-driven technology solutions towards accelerating scientific research in medicine and ensure that data from biomedical publications can be maximally leveraged and reach a wide range of biomedical researchers.*

*Each phase of the competition is designed to spur innovation in the field of natural language processing, asking competitors to design systems that can accurately recognize scientific concepts from the text of scientific articles, connect those concepts into knowledge assertions, and determine if that claim is a novel finding or background information.*

>Part 1: Given only an abstract text, the goal is to find all the nodes or biomedical entities (position in text and BioLink Model Category).    
>Part 2: Given the abstract and the nodes annotated from it, the goal is to find all the relationships between them (pair of nodes, BioLink Model Predicate and novelty). 

**The goal of the first part of the LitCoin NLP Challenge is to identify the position and type of biomedical concepts (entities) mentions within a research paper’s title and abstract.**

In [7]:
import pandas as pd
import numpy as np
import spacy as sp

In [8]:
folder = 'C:/Users/saaye/OneDrive/Documents/Machine Learning Projects/LitCoin NLP Challenge/data/'

abstract_test = pd.read_csv(folder + 'abstracts_test.csv', sep='\t')
abstract_train = pd.read_csv(folder + 'abstracts_train.csv', sep='\t')
entities_train = pd.read_csv(folder + 'entities_train.csv', sep='\t')
relations_train = pd.read_csv(folder + 'relations_train.csv', sep='\t')
submission_example = pd.read_csv(folder + 'submission_example.csv', sep='\t')

In [3]:
print(f'Abstract training dataset has {abstract_train.shape[0]} columns and {abstract_train.shape[1]} rows.')
print(f'Relations training dataset has {relations_train.shape[0]} columns and {relations_train.shape[1]} rows.')
print(f'Entities training dataset has {entities_train.shape[0]} columns and {entities_train.shape[1]} rows.')

Abstract training dataset has 400 columns and 3 rows.
Relations training dataset has 4280 columns and 6 rows.
Entities training dataset has 13636 columns and 7 rows.


In [4]:
print(f'An example of an observation is as follows: \n # abstract_id: PubMed ID of the research paper. : {abstract_train["abstract_id"][0]}')
print(f' # title: title of the research paper. : {abstract_train["title"][0]}')
print(f' # abstract: abstract or summary of the research paper. : {abstract_train["abstract"][0]}')

An example of an observation is as follows: 
 # abstract_id: PubMed ID of the research paper. : 1353340
 # title: title of the research paper. : Late-onset metachromatic leukodystrophy: molecular pathology in two siblings.
 # abstract: abstract or summary of the research paper. : We report on a new allele at the arylsulfatase A (ARSA) locus causing late-onset metachromatic leukodystrophy (MLD). In that allele arginine84, a residue that is highly conserved in the arylsulfatase gene family, is replaced by glutamine. In contrast to alleles that cause early-onset MLD, the arginine84 to glutamine substitution is associated with some residual ARSA activity. A comparison of genotypes, ARSA activities, and clinical data on 4 individuals carrying the allele of 81 patients with MLD examined, further validates the concept that different degrees of residual ARSA activity are the basis of phenotypical variation in MLD.. 


In [9]:
df = pd.DataFrame(abstract_train['title'] + ' ' + abstract_train['abstract'], columns=['text'])

The position of a biomedical entity's mention in the text is determined by two ‘offset’ numbers: ‘offset_start’ and ‘offset_finish’, which indicate the index of the character where a given mention substring begins and ends, respectively. The input string considered for these indices is the concatenation of the title and abstract strings in the following manner: string = title + ' ' + abstract (i.e., there is one extra character between the 2 when accounting for the offset). 

In [10]:
pattern1 = (set(entities_train[entities_train['type'] == 'DiseaseOrPhenotypicFeature']['mention']))
pattern2 = (set(entities_train[entities_train['type'] == 'ChemicalEntity']['mention']))
pattern3 = (set(entities_train[entities_train['type'] == 'OrganismTaxon']['mention']))
pattern4 = (set(entities_train[entities_train['type'] == 'GeneOrGeneProduct']['mention']))
pattern5 = (set(entities_train[entities_train['type'] == 'SequenceVariant']['mention']))
pattern6 = (set(entities_train[entities_train['type'] == 'CellLine']['mention']))

pattern = [{'label': 'DiseaseOrPhenotypicFeature', 'pattern': pattern1},
           {'label': 'ChemicalEntity', 'pattern': pattern2},
           {'label': 'OrganismTaxon', 'pattern': pattern3},
           {'label': 'GeneOrGeneProduct', 'pattern': pattern4},
           {'label': 'SequenceVariant', 'pattern': pattern5},
           {'label': 'CellLine', 'pattern': pattern6}]

In [23]:
text = df['text'].str.cat(sep=' ')

In [49]:
patterns = [{"label": "DiseaseOrPhenotypicFeature", "pattern": str(i)} for i in pattern1]

In [77]:
nlp = sp.blank("en")
ruler = nlp.add_pipe("entity_ruler")
patterns = entities_train[["type", "mention"]].rename(columns={"type": "label", "mention": "pattern"}).to_dict("records")
ruler.add_patterns(patterns)
doc = nlp(df['text'][0])
print([(ent.text, ent.label_) for ent in doc.ents])

[('metachromatic leukodystrophy', 'DiseaseOrPhenotypicFeature'), ('arylsulfatase A', 'GeneOrGeneProduct'), ('ARSA', 'GeneOrGeneProduct'), ('metachromatic leukodystrophy', 'DiseaseOrPhenotypicFeature'), ('MLD', 'DiseaseOrPhenotypicFeature'), ('arginine84', 'SequenceVariant'), ('arylsulfatase', 'GeneOrGeneProduct'), ('glutamine', 'ChemicalEntity'), ('MLD', 'DiseaseOrPhenotypicFeature'), ('arginine84 to glutamine', 'SequenceVariant'), ('ARSA', 'GeneOrGeneProduct'), ('ARSA', 'GeneOrGeneProduct'), ('patients', 'OrganismTaxon'), ('MLD', 'DiseaseOrPhenotypicFeature'), ('ARSA', 'GeneOrGeneProduct'), ('MLD', 'DiseaseOrPhenotypicFeature')]


In [82]:
entities_train[entities_train["abstract_id"]==1353340]

,id,abstract_id,offset_start,offset_finish,type,mention,entity_ids
0,0,1353340,11,39,DiseaseOrPhenotypicFeature,metachromatic leukodystrophy,D007966
1,1,1353340,111,126,GeneOrGeneProduct,arylsulfatase A,410
2,2,1353340,128,132,GeneOrGeneProduct,ARSA,410
3,3,1353340,159,187,DiseaseOrPhenotypicFeature,metachromatic leukodystrophy,D007966
4,4,1353340,189,192,DiseaseOrPhenotypicFeature,MLD,D007966
5,5,1353340,210,220,SequenceVariant,arginine84,rs74315458
6,6,1353340,264,277,GeneOrGeneProduct,arylsulfatase,410
7,7,1353340,363,366,DiseaseOrPhenotypicFeature,MLD,D007966
8,8,1353340,372,395,SequenceVariant,arginine84 to glutamine,rs74315458
9,9,1353340,442,446,GeneOrGeneProduct,ARSA,410


In [83]:
len([(ent.text, ent.label_) for ent in doc.ents])

16

In [78]:
sp.displacy.render(doc, style="ent", jupyter=True)

In [65]:
[{"label": "DiseaseOrPhenotypicFeature", "pattern": str(i)} for i in pattern1]

[{'label': 'DiseaseOrPhenotypicFeature',
  'pattern': 'metachromatic leukodystrophy'},
 {'label': 'DiseaseOrPhenotypicFeature', 'pattern': 'MLD'},
 {'label': 'DiseaseOrPhenotypicFeature', 'pattern': 'phenylketonuria'},
 {'label': 'DiseaseOrPhenotypicFeature',
  'pattern': 'Classical phenylketonuria'},
 {'label': 'DiseaseOrPhenotypicFeature',
  'pattern': 'autosomal recessive disease'},
 {'label': 'DiseaseOrPhenotypicFeature',
  'pattern': 'deficiency of hepatic phenylalanine hydroxylase'},
 {'label': 'DiseaseOrPhenotypicFeature', 'pattern': 'PKU'},
 {'label': 'DiseaseOrPhenotypicFeature', 'pattern': 'PAH deficiencies'},
 {'label': 'DiseaseOrPhenotypicFeature', 'pattern': 'hypokalemia'},
 {'label': 'DiseaseOrPhenotypicFeature', 'pattern': 'hypotensive'},
 {'label': 'DiseaseOrPhenotypicFeature', 'pattern': 'hypertensive'},
 {'label': 'DiseaseOrPhenotypicFeature', 'pattern': 'stroke'},
 {'label': 'DiseaseOrPhenotypicFeature', 'pattern': 'hypotension'},
 {'label': 'DiseaseOrPhenotypicFeatu

In [75]:
patterns = entities_train[["type", "mention"]].rename(columns={"type": "label", "mention": "pattern"}).to_dict("records")